In [1]:
import pandas as pd

С использованием библиотеки https://github.com/natasha выполним базовые задачи NLP для русского языка:
- токенизацию, 
- сегментирование предложений, 
- встраивание слов, 
- тегирование морфологии, 
- лемматизацию, 
- нормализацию фраз, 
- синтаксический анализ, 
- тегирование NER, 
- извлечение фактов.

# Загружаем текст

In [2]:
f=open("..\\dataset\\1968.txt", "r")
if f.mode == 'r':
    contents =f.read()

In [3]:
# contents

In [4]:
contents = contents.replace('\n\n',' ')

In [5]:
len(contents)

65687

# Инициализация

In [6]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

In [7]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

doc = Doc(contents)

# Сегментация текста

In [8]:
doc.segment(segmenter)

In [10]:
display(doc.tokens[:5])

[DocToken(stop=9, text='Уважаемые'),
 DocToken(start=10, stop=18, text='граждане'),
 DocToken(start=19, stop=25, text='России'),
 DocToken(start=25, stop=26, text='!'),
 DocToken(start=27, stop=36, text='Уважаемые')]

In [11]:
display(doc.sents[:5])

[DocSent(stop=26, text='Уважаемые граждане России!', tokens=[...]),
 DocSent(start=27, stop=71, text='Уважаемые депутаты и члены Совета Федерации!', tokens=[...]),
 DocSent(start=72, stop=123, text='Начну своё Послание с оценки событий текущего год..., tokens=[...]),
 DocSent(start=124, stop=201, text='В 2008 году в нашей стране произошло обновление к..., tokens=[...]),
 DocSent(start=202, stop=269, text='По итогам выборов Президента было сформировано но..., tokens=[...])]

In [12]:
len(doc.sents)

678

# Морфология

In [13]:
doc.tag_morph(morph_tagger)

In [14]:
display(doc.tokens[:5])

[DocToken(stop=9, text='Уважаемые', pos='ADJ', feats=<Nom,Pos,Plur>),
 DocToken(start=10, stop=18, text='граждане', pos='NOUN', feats=<Anim,Nom,Masc,Plur>),
 DocToken(start=19, stop=25, text='России', pos='PROPN', feats=<Inan,Gen,Fem,Sing>),
 DocToken(start=25, stop=26, text='!', pos='PUNCT'),
 DocToken(start=27, stop=36, text='Уважаемые', pos='ADJ', feats=<Nom,Pos,Plur>)]

In [15]:
doc.sents[0].morph.print()

           Уважаемые ADJ|Case=Nom|Degree=Pos|Number=Plur
            граждане NOUN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Plur
              России PROPN|Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
                   ! PUNCT


In [16]:
doc.sents[1].morph.print()

           Уважаемые ADJ|Case=Nom|Degree=Pos|Number=Plur
            депутаты NOUN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Plur
                   и CCONJ
               члены NOUN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Plur
              Совета PROPN|Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing
           Федерации PROPN|Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
                   ! PUNCT


# LEMMA

In [17]:
for token in doc.tokens:
    token.lemmatize(morph_vocab)

In [18]:
display(doc.tokens[:5])

[DocToken(stop=9, text='Уважаемые', pos='ADJ', feats=<Nom,Pos,Plur>, lemma='уважаемый'),
 DocToken(start=10, stop=18, text='граждане', pos='NOUN', feats=<Anim,Nom,Masc,Plur>, lemma='гражданин'),
 DocToken(start=19, stop=25, text='России', pos='PROPN', feats=<Inan,Gen,Fem,Sing>, lemma='россия'),
 DocToken(start=25, stop=26, text='!', pos='PUNCT', lemma='!'),
 DocToken(start=27, stop=36, text='Уважаемые', pos='ADJ', feats=<Nom,Pos,Plur>, lemma='уважаемый')]

In [19]:
text = []
pos = []
lemma = []
for i in range(len(doc.tokens)):
    text.append(doc.tokens[i].text)
    pos.append(doc.tokens[i].pos)
    lemma.append(doc.tokens[i].lemma)
lemma_df = pd.DataFrame()
lemma_df['text'] = text
lemma_df['pos'] = pos
lemma_df['lemma'] = lemma

In [20]:
lemma_df

,text,pos,lemma
0,Уважаемые,ADJ,уважаемый
1,граждане,NOUN,гражданин
2,России,PROPN,россия
3,!,PUNCT,!
4,Уважаемые,ADJ,уважаемый
5,депутаты,NOUN,депутат
6,и,CCONJ,и
7,члены,NOUN,член
8,Совета,PROPN,совет
9,Федерации,PROPN,федерация


In [21]:
lemma_df.describe()

,text,pos,lemma
count,10074,10074,10074
unique,3676,15,2097
top,",",NOUN,","
freq,687,2609,687


In [22]:
grouped = lemma_df.groupby('lemma')

In [23]:
grouped = grouped.count()

In [24]:
dict(grouped)['text']['экономика']

20

In [25]:
grouped_dict = dict(grouped)
frecuency_lict = []
for i in range(len(lemma_df)):
    i_lemma = lemma_df['lemma'][i]
    frecuency_lict.append(grouped_dict['text'][i_lemma])

In [26]:
lemma_df['freq'] = frecuency_lict

In [28]:
lemma_df_NOUN = lemma_df[lemma_df['pos']=='NOUN']
lemma_df_NOUN.sort_values(by='freq', ascending=False)[['lemma','freq']].drop_duplicates()[:10]

,lemma,freq
5415,человек,42
5102,система,36
5363,гражданин,33
630,страна,33
7542,мера,28
7131,год,28
3095,развитие,28
7806,государство,28
4141,решение,26
5833,право,26


In [29]:
lemma_df_ADJ = lemma_df[lemma_df['pos']=='ADJ']
lemma_df_ADJ.sort_values(by='freq', ascending=False)[['lemma','freq']].drop_duplicates()[:10]

,lemma,freq
198,весь,82
2295,должный,56
5679,новый,46
4337,государственный,39
12,свой,34
5100,российский,29
8383,самый,29
9657,нужный,24
9571,другой,21
8960,международный,21


In [30]:
lemma_df_VERB = lemma_df[lemma_df['pos']=='VERB']
lemma_df_VERB.sort_values(by='freq', ascending=False)[['lemma','freq']].drop_duplicates()[:10]

,lemma,freq
1934,быть,115
871,мочь,15
8404,хотеть,15
6963,стать,15
9269,считать,15
758,сказать,15
7757,принять,13
1692,получить,12
5835,знать,12
608,сделать,11


# SYNTAX

In [31]:
doc.parse_syntax(syntax_parser)

In [32]:
display(doc.tokens[:5])

[DocToken(stop=9, text='Уважаемые', id='1_1', head_id='1_2', rel='amod', pos='ADJ', feats=<Nom,Pos,Plur>, lemma='уважаемый'),
 DocToken(start=10, stop=18, text='граждане', id='1_2', head_id='1_2', rel='nsubj', pos='NOUN', feats=<Anim,Nom,Masc,Plur>, lemma='гражданин'),
 DocToken(start=19, stop=25, text='России', id='1_3', head_id='1_2', rel='nmod', pos='PROPN', feats=<Inan,Gen,Fem,Sing>, lemma='россия'),
 DocToken(start=25, stop=26, text='!', id='1_4', head_id='1_2', rel='punct', pos='PUNCT', lemma='!'),
 DocToken(start=27, stop=36, text='Уважаемые', id='2_1', head_id='2_2', rel='amod', pos='ADJ', feats=<Nom,Pos,Plur>, lemma='уважаемый')]

In [33]:
doc.sents[0].syntax.print()

    ┌► Уважаемые amod
┌─┌─└─ граждане  
│ └──► России    nmod
└────► !         punct


In [34]:
display(doc.tokens[:10])

[DocToken(stop=9, text='Уважаемые', id='1_1', head_id='1_2', rel='amod', pos='ADJ', feats=<Nom,Pos,Plur>, lemma='уважаемый'),
 DocToken(start=10, stop=18, text='граждане', id='1_2', head_id='1_2', rel='nsubj', pos='NOUN', feats=<Anim,Nom,Masc,Plur>, lemma='гражданин'),
 DocToken(start=19, stop=25, text='России', id='1_3', head_id='1_2', rel='nmod', pos='PROPN', feats=<Inan,Gen,Fem,Sing>, lemma='россия'),
 DocToken(start=25, stop=26, text='!', id='1_4', head_id='1_2', rel='punct', pos='PUNCT', lemma='!'),
 DocToken(start=27, stop=36, text='Уважаемые', id='2_1', head_id='2_2', rel='amod', pos='ADJ', feats=<Nom,Pos,Plur>, lemma='уважаемый'),
 DocToken(start=37, stop=45, text='депутаты', id='2_2', head_id='2_2', rel='appos', pos='NOUN', feats=<Anim,Nom,Masc,Plur>, lemma='депутат'),
 DocToken(start=46, stop=47, text='и', id='2_3', head_id='2_4', rel='cc', pos='CCONJ', lemma='и'),
 DocToken(start=48, stop=53, text='члены', id='2_4', head_id='2_2', rel='conj', pos='NOUN', feats=<Anim,Nom,Masc

# NER

In [35]:
doc.tag_ner(ner_tagger)

In [36]:
display(doc.spans[:5])

[DocSpan(start=19, stop=25, type='LOC', text='России', tokens=[...]),
 DocSpan(start=54, stop=70, type='ORG', text='Совета Федерации', tokens=[...]),
 DocSpan(start=255, stop=268, type='ORG', text='Правительство', tokens=[...]),
 DocSpan(start=303, stop=323, type='ORG', text='Государственной Думе', tokens=[...]),
 DocSpan(start=847, stop=853, type='LOC', text='России', tokens=[...])]

In [37]:
doc.ner.print()

Уважаемые граждане России! Уважаемые депутаты и члены Совета 
                   LOC───                             ORG────
Федерации! Начну своё Послание с оценки событий текущего года. В 2008 
─────────                                                             
году в нашей стране произошло обновление ключевых властных институтов.
 По итогам выборов Президента было сформировано новое Правительство. В
                                                      ORG──────────   
 полную силу заработали в новой Государственной Думе парламентские 
                                ORG─────────────────               
партии. Началась реализация новых планов долгосрочного развития 
экономики и социальной сферы. Строятся заводы и дороги. 
Перевооружаются армия и флот. Осваиваются новые технологии. Создаются 
учебные, научные, медицинские, центры. Наши спортсмены показывают 
примеры ярких побед. Но этот год стал для наших граждан, для всех нас 
не только временем новых надежд и достижений. Произошл

# PHRASE NORM

In [38]:
for span in doc.spans:
    span.normalize(morph_vocab)

In [39]:
display(doc.spans[:5])

[DocSpan(start=19, stop=25, type='LOC', text='России', tokens=[...], normal='Россия'),
 DocSpan(start=54, stop=70, type='ORG', text='Совета Федерации', tokens=[...], normal='Совет Федерации'),
 DocSpan(start=255, stop=268, type='ORG', text='Правительство', tokens=[...], normal='Правительство'),
 DocSpan(start=303, stop=323, type='ORG', text='Государственной Думе', tokens=[...], normal='Государственная Дума'),
 DocSpan(start=847, stop=853, type='LOC', text='России', tokens=[...], normal='Россия')]

In [40]:
{_.text: _.normal for _ in doc.spans if _.text != _.normal}

{'России': 'Россия',
 'Совета Федерации': 'Совет Федерации',
 'Государственной Думе': 'Государственная Дума',
 'Южной Осетии': 'Южная Осетия',
 'Кавказе': 'Кавказ',
 'Чёрное море': 'Черное море',
 'Европе': 'Европа',
 'Соединённых Штатов': 'Соединенных Штаты',
 'Советского Союза': 'Советский Союз',
 'Соединёнными Штатами Америки': 'Соединенные Штаты Америка',
 'Южном Кавказе': 'Южный Кавказ',
 'Федеральному Собранию': 'Федеральное Собрание',
 'Российского государства': 'Российское государство',
 'Абхазии': 'Абхазия',
 'Россию': 'Россия',
 'Россией': 'Россия',
 'Правительства': 'Правительство',
 'Федерации': 'Федерация',
 'Банка России': 'Банк России',
 'Петра Столыпина': 'Петр Столыпин',
 'Руси': 'Русь',
 'Госдуму': 'Госдума',
 'Государственную Думу': 'Государственная Дума',
 'Совете Федерации': 'Совет Федерации',
 'Общественной палаты': 'Общественная палата',
 'Государственной Думы': 'Государственная Дума',
 'Президенту': 'Президент',
 'Федерального Собрания': 'Федеральное Собрание',


# Fact extraction

In [41]:
for span in doc.spans:
    if span.type == PER:
        span.extract_fact(names_extractor)
    
{_.normal: _.fact.as_dict for _ in doc.spans if _.fact}

{'Петр Столыпин': {'first': 'Петр', 'last': 'Столыпин'},
 'Николай Коркунов': {'first': 'Николай', 'last': 'Коркунов'},
 'Борис Чичерин': {'first': 'Борис', 'last': 'Чичерин'},
 'Василий Леонтьев': {'first': 'Василий', 'last': 'Леонтьев'}}

# DatesExtractor

In [42]:
list(dates_extractor(contents))

[Match(
     start=126,
     stop=135,
     fact=Date(
         year=2008,
         month=None,
         day=None
     )
 ), Match(
     start=3526,
     stop=3535,
     fact=Date(
         year=2008,
         month=None,
         day=None
     )
 ), Match(
     start=11264,
     stop=11272,
     fact=Date(
         year=2020,
         month=None,
         day=None
     )
 ), Match(
     start=16942,
     stop=16951,
     fact=Date(
         year=1993,
         month=None,
         day=None
     )
 ), Match(
     start=20954,
     stop=20963,
     fact=Date(
         year=2007,
         month=None,
         day=None
     )
 ), Match(
     start=37378,
     stop=37396,
     fact=Date(
         year=2009,
         month=1,
         day=1
     )
 ), Match(
     start=46657,
     stop=46666,
     fact=Date(
         year=2010,
         month=None,
         day=None
     )
 ), Match(
     start=54342,
     stop=54351,
     fact=Date(
         year=2010,
         month=None,
         day=Non